<a href="https://colab.research.google.com/github/hasanzeynal/Streamlit-Car-Price/blob/main/Modeling/car_price_prediction_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import tensorflow as tf
import pandas as pd

In [ ]:
miau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/test.csv')
train_df.head(),test_df.head()

(         ID  Price  Levy Manufacturer    Model  Prod. year   Category  \
 0  45654403  13328  1399        LEXUS   RX 450        2010       Jeep   
 1  44731507  16621  1018    CHEVROLET  Equinox        2011       Jeep   
 2  45774419   8467     -        HONDA      FIT        2006  Hatchback   
 3  45769185   3607   862         FORD   Escape        2011       Jeep   
 4  45809263  11726   446        HONDA      FIT        2014  Hatchback   
 
   Leather interior Fuel type Engine volume    Mileage  Cylinders  \
 0              Yes    Hybrid           3.5  186005 km        6.0   
 1               No    Petrol             3  192000 km        6.0   
 2               No    Petrol           1.3  200000 km        4.0   
 3              Yes    Hybrid           2.5  168966 km        4.0   
 4              Yes    Petrol           1.3   91901 km        4.0   
 
   Gear box type Drive wheels   Doors             Wheel   Color  Airbags  
 0     Automatic          4x4  04-May        Left wheel  Silver

In [ ]:
test_df.isna().sum(),train_df.isna().sum()

(ID                     0
 Levy                   0
 Manufacturer           0
 Model                  0
 Prod. year             0
 Category               0
 Leather interior       0
 Fuel type              0
 Engine volume          0
 Mileage                0
 Cylinders              0
 Gear box type          0
 Drive wheels           0
 Doors                  0
 Wheel                  0
 Color                  0
 Airbags                0
 Price               8245
 dtype: int64,
 ID                  0
 Price               0
 Levy                0
 Manufacturer        0
 Model               0
 Prod. year          0
 Category            0
 Leather interior    0
 Fuel type           0
 Engine volume       0
 Mileage             0
 Cylinders           0
 Gear box type       0
 Drive wheels        0
 Doors               0
 Wheel               0
 Color               0
 Airbags             0
 dtype: int64)